In [0]:
#Import statements
import re
import json
import random
import hidateinfer
import numpy as np
from pyspark.sql import functions as F
from pyspark.sql.functions import col, lower, udf, count, when, round, size, lit, row_number, length, translate, expr, sum as sum_, split, explode, collect_list, desc, broadcast
from pyspark.sql.types import StringType, BooleanType, DateType, DecimalType, DoubleType, ArrayType
from pyspark.sql.window import Window
from pyspark.sql import DataFrame
from collections import defaultdict, Counter
from operator import add
from functools import reduce
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient


In [0]:
%run ./utils/util_funcs

In [0]:
import datetime
import decimal
def myconverter(obj):
    if isinstance(obj, np.integer):
        return int(obj)
    elif isinstance(obj, np.floating):
        return float(obj)
    elif isinstance(obj, np.ndarray):
        return obj.tolist()
    elif isinstance(obj, np.bool_):
        return bool(obj)
    elif isinstance(obj, (datetime.datetime, datetime.date)):
        return obj.isoformat()
    elif isinstance(obj, decimal.Decimal):
        return str(obj)

In [0]:
import pandas as pd
import numpy as np
import pandasql as psql

# import seaborn as sns
# sns.set_style('whitegrid')
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_columns' , None)
pd.set_option('display.max_rows' , None)
pd.set_option('display.max_colwidth' , None)
pd.set_option('max_info_columns' , -1)
pd.set_option('max_info_rows' , -1)

# import warnings
# from pandas.core.common import SettingWithCopyWarning
# warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

import logging
logging.getLogger("py4j").setLevel(logging.ERROR)
from collections import defaultdict,Counter
import random

#added by sahil
import string
import os
from dataprep.clean import *
import zipcodes
from uszipcode import SearchEngine
engine = SearchEngine()

In [0]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient

dbutils.widgets.text("container_name","none",label="container_name")
dbutils.widgets.text("AZURE_STORAGE_CONNECTION_STRING","none",label="AZURE_STORAGE_CONNECTION_STRING")
dbutils.widgets.text("filename","none",label="filename")

In [0]:
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
import json
import io

# Get the values from the widgets
filename = dbutils.widgets.get("filename")
que = dbutils.widgets.get("que")

destination_storage_type = dbutils.widgets.get("destination_storage_type")
if destination_storage_type == 'ABS':
    CONTAINER_NAME = dbutils.widgets.get("container_name")
    AZURE_STORAGE_CONNECTION_STRING = dbutils.widgets.get("AZURE_STORAGE_CONNECTION_STRING")
elif destination_storage_type == 'S3':
    storage_access_key = dbutils.widgets.get("storage_access_key")
    storage_secret_key = dbutils.widgets.get("storage_secret_key")
    bucket_name = dbutils.widgets.get("bucket_name")
else:
    print(f"Invalid destination_storage_type {destination_storage_type}")

if destination_storage_type == 'ABS':
    # Create a BlobServiceClient object using the connection string
    BLOB_SERVICE_CLIENT = BlobServiceClient.from_connection_string(AZURE_STORAGE_CONNECTION_STRING)


filename_zeroshot1 = filename + '.zeroshot'
filename_zeroshot2 = filename + '.zeroshot2'
print(filename)
print(filename_zeroshot1)
print(filename_zeroshot2)

In [0]:
def infer_datatypes(df, column_name):
    """
    Infer the data types of a specified column in a PySpark DataFrame.

    Args:
        df (DataFrame): The PySpark DataFrame to analyze.
        column_name (str): The name of the column to analyze.

    Returns:
        dict: A dictionary containing the counts of StringType, DateType, and Numeric values,
              along with the majority data type.

    """
    str_count = 0
    date_count = 0
    num_count = 0

    if column_name in df.columns:
        column_type = df.schema[column_name].dataType
        if column_type == StringType():
            str_count = df.filter(F.col(column_name).isNotNull()).count()
        elif column_type == DateType():
            date_count = df.filter(F.col(column_name).isNotNull()).count()
        else:
            num_count = df.filter(F.col(column_name).isNotNull()).count()

    data_types_dict = {
        "Non-Numeric": str_count,
        "Date": date_count,
        "Numeric": num_count,
        "Majority Type": get_majority_type(str_count, date_count, num_count)
    }

    return data_types_dict

def get_majority_type(str_count, date_count, num_count):
    """
    Determine the majority data type based on the counts of StringType, DateType, and Numeric values.

    Args:
        str_count (int): The count of StringType values.
        date_count (int): The count of DateType values.
        num_count (int): The count of Numeric values.

    Returns:
        str: The majority data type ("String", "Date", "Numeric", or "Unknown" if there's a tie).

    """
    if str_count > date_count and str_count > num_count:
        return "Non-Numeric"
    elif date_count > str_count and date_count > num_count:
        return "Date"
    elif num_count > str_count and num_count > date_count:
        return "Numeric"
    else:
        return "Unknown"


In [0]:
def replace_only_special_chars(text):
    if all(not i.isalnum() for i in text):
        return np.nan
    else:
        return text

def filter_blank(x):
    if ''.join(set(str(x))) in ["", " "]:
        return True
    else:
        return False

class GlobalFuncs():
    global get_num_cols
    def get_num_cols(df):
        new = df.copy()
        num_cols = []
        new = new.astype(str)
        for i in new.columns:
            new[i] = new[i].apply(lambda x : replace_only_special_chars(x)).astype(str)
            try:
                if new[i].astype(float).isnull().sum() != len(new):
                    new[i] = new[i].apply(lambda x : float(x.strip().replace(",","")))
                    n = int(len(new[i].dropna())/5)
                    if abs(new[i].sample(n).mean()) >=0:
                        #print(i, df[i].sample(n).mean())
                        num_cols.append(i)
            except:
                pass
        return new, num_cols

    
    global type_casting
    def type_casting(df, dtypes, num=True, non_num=True, date=True):
        df = df.reset_index(drop=True)
        #df = df.astype(str)
        if num==True:
            for i in dtypes["Numeric"]:
                df[i] = df[i].astype(str).apply(lambda x : float(x.strip().replace(",","")))
        if date==True:
            for i in dtypes["Date"]:
                df[i] = df[i].astype(str).apply(lambda x : x.strip())
                df.loc[df[i].isin(["nan", "NAN", "NaN", "Nan", "NaT"]), i] = np.nan
                df[i] = pd.to_datetime(df[i], errors ='coerce')
        if non_num ==True:
            for i in dtypes["Non-Numeric"]:
                df[i] = df[i].astype(str).apply(lambda x : x.strip())
                df.loc[df[i].isin(["nan", "NAN", "NaN", "Nan"]), i] = np.nan
        return df

In [0]:
import re

def remove_special_char(word):
    final = re.sub(r"[^a-zA-Z0-9.]+", ' ', word)
    final = re.sub("\s\s+", " ", final)
    return final.strip()
    
def check_special_char(word):
    #word = re.sub(r'([^\s])\s([^\s])', r'\1_\2', word)
    #whitespaces = re.findall(r"\s+", word)
    sp_chars = [c for c in word if not c.isalnum()]
    sp_chars = [y for y in sp_chars if y != " "]
    #sp_chars = sp_chars + spaces
    #unique_sp_chars = list(set(sp_chars))
    return sp_chars

def check_whitespaces(word):
    whitespaces = re.findall(r"\s+", word)
    return whitespaces
  
def check_non_ascii(word):
    non_ascii_chars = [i for i in word if not i.isascii()]
    return non_ascii_chars
  
def check_ascii(word):
    #word = word.replace(" ", "")
    ascii_chars = [c for c in word if c.isascii() and not c.isspace()]
    return ascii_chars
    
def check_alnum(word):
    alnum_chars = [c for c in word if c.isalnum() and c.isascii()]
    return alnum_chars
    
def check_all_char(word):
    
    len_sp_chars = len(check_special_char(word))
    len_whitespaces = len(check_whitespaces(word))
    len_non_ascii_chars = len(check_non_ascii(word))
    len_alnum_chars = len(check_alnum(word))
    
    return len_sp_chars + len_whitespaces + len_non_ascii_chars + len_alnum_chars

In [0]:
def mode(ndarray, axis=0):
    # Check inputs
    #ndarray = np.asarray(ndarray)
    ndim = ndarray.ndim
    if ndarray.size == 1:
        return (ndarray[0], 1)
    elif ndarray.size == 0:
        #raise Exception('Cannot compute mode on empty array')
        return ([], [])
    try:
        axis = range(ndarray.ndim)[axis]
    except:
        raise Exception('Axis "{}" incompatible with the {}-dimension array'.format(axis, ndim))

    # If array is 1-D and np version is > 1.9 np.unique will suffice
    #if all([ndim == 1,
            #int(np.__version__.split('.')[0]) >= 1,
            #int(np.__version__.split('.')[1]) >= 9]):
    if ndim ==1:
        modals, counts = np.unique(ndarray, return_counts=True)
        index = np.argmax(counts) #counts.argmax()
        return modals[index], counts[index]

    # Sort array
    sort = np.sort(ndarray, axis=axis)
    # Create array to transpose along the axis and get padding shape
    transpose = np.roll(np.arange(ndim)[::-1], axis)
    shape = list(sort.shape)
    shape[axis] = 1
    # Create a boolean array along strides of unique values
    strides = np.concatenate([np.zeros(shape=shape, dtype='bool'),
                                 np.diff(sort, axis=axis) == 0,
                                 np.zeros(shape=shape, dtype='bool')],
                                axis=axis).transpose(transpose).ravel()
    # Count the stride lengths
    counts = np.cumsum(strides)
    counts[~strides] = np.concatenate([[0], np.diff(counts[~strides])])
    counts[strides] = 0
    # Get shape of padded counts and slice to return to the original shape
    shape = np.array(sort.shape)
    shape[axis] += 1
    shape = shape[transpose]
    slices = [slice(None)] * ndim
    slices[axis] = slice(1, None)
    # Reshape and compute final counts
    counts = counts.reshape(shape).transpose(transpose)[slices] + 1

    # Find maximum counts and return modals/counts
    slices = [slice(None, i) for i in sort.shape]
    del slices[axis]
    index = np.ogrid[slices]
    index.insert(axis, np.argmax(counts, axis=axis))
    return sort[index], counts[index]

from scipy import stats as st
import pylab as pl

class AllStats():
    def __init__(self,yourarray):
        v=yourarray
        self.mini= float(min(v))
        self.q1 = float(np.percentile(v, 25))
        self.q2 = float(np.percentile(v, 50))
        self.q3 = float(np.percentile(v, 75))
        self.maxi= float(max(v))
        self.mean= float(np.mean(v))
        self.median= float(np.median(v))
        self.mode= st.mode(v)[0][0]
        self.count= float(v.shape[0])
        self.range= float(max(v)-min(v))
        self.iqr = float( np.percentile(v, 75)-np.percentile(v, 25) )
        self.std= float(np.std(v))
        self.mad = float(st.median_abs_deviation(v))
        self.var= float(np.var(v))
        self.coeffvar= float(((np.std(v))/(np.mean(v)))*100)
        self.skewness = float(st.skew(v))
        self.kurtosis = float(st.kurtosis(v))
        
    def summarystats(self, v):
        mylist= [self.mini, self.q1, self.q2, self.q3, self.maxi, self.mean, self.median, self.mode, self.count,
                 self.range, self.iqr, self.std, self.mad, self.var, self.coeffvar, self.skewness, self.kurtosis]
        d= {'Summary Statistics': mylist}
        listnames = ['Min', 'First Quartile: Q1 (25%)', 'Second Quartile: Q2 (50%)', 'Third Quartile: Q3 (75%)', 'Max',
                    'Mean', 'Median', 'Mode', 'Count', 'Range', 'IQR', 'StdDev', 'MAD', 'Variance', 'Coeff. of Variation',
                    'Skewness', 'Kurtosis']
        return ( pd.DataFrame(data= d, index= listnames) )  
    
    def outliers_zscore(self,v):
        threshold =2 #3.0
        self.zscores = [(i- self.mean)/self.std for i in v]
        self.outliers = [v[i] for i in range(len(self.zscores)) if np.abs(self.zscores[i]) > threshold]
        return self
    
    def outliers_modified_zscore(self,v):
        #ST DEV ≈ 1.155 MAD (uniform dist)
        #ST DEV ≈ 1.254 MAD for small samples (MAD <2.5) and 1.4826 MAD for large samples (MAD>=2.5) (normal dist)
        threshold = 2.508 #3.762
        median_y = self.median
        median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in v])
        self.modified_z_scores = [0.7974 * (y - median_y) / median_absolute_deviation_y for y in v] #0.6745
        self.outliers = [v[i] for i in range(len(self.modified_z_scores)) if np.abs(self.modified_z_scores[i]) > threshold]
        return self
    
    def outliers_iqr(self,v):
        quartile_1, quartile_3 = np.percentile(v, [25, 75])
        iqr = quartile_3 - quartile_1
        lower_bound = quartile_1 - (iqr * 1.5)
        upper_bound = quartile_3 + (iqr * 1.5)
        self.outliers= np.where((v > upper_bound) | (v < lower_bound))
        return self

    def outliers_std(self,v):
        #ST DEV ≈ 1.155 MAD (uniform dist)
        #ST DEV ≈ 1.254 MAD for small samples (MAD <2.5) and 1.4826 MAD for large samples (MAD>=2.5) (normal dist)
        self.outliersmean=[]
        self.outliersmedian=[]
        median_y = self.median
        median_absolute_deviation_y = np.median([np.abs(y - median_y) for y in v])
        for x in v:
            if ( (x < (self.mean - (2 * self.std))) or (x > (self.mean + (2 * self.std))) ):
                self.outliersmean.append(x)
            if ( (x < (self.median - (2.508 * median_absolute_deviation_y))) or (x > (self.median + (2.508 * median_absolute_deviation_y))) ):
            #if ( (x < (self.median - (2 * self.std))) or (x > (self.median + (2 * self.std))) ):
                self.outliersmedian.append(x)
            else:
                continue
        return self
    
        
    def distribution_mean(self, v):
        #v = sorted(v)
        fit = st.norm.pdf(v, self.mean, self.std)
        pl.plot(v,fit,'--')
        pl.hist(v,density=True)
        plt.axvline( (self.mean - 2*(self.std) ), color= 'g')
        plt.axvline( (self.mean + 2*(self.std) ), color= 'g')
        pl.show()
        

    def distribution_med(self, v):
        #v = sorted(v)
        fit = st.norm.pdf(v,self.median, self.std)
        pl.plot(v,fit,'--')
        pl.hist(v,density=True)
        plt.axvline( (self.median - (2.508 * median_absolute_deviation_y)), color= 'g')
        plt.axvline( (self.median + (2.508 * median_absolute_deviation_y)), color ='g')
        pl.show()

In [0]:
que = dbutils.widgets.get("que")
sdf = sqlContext.sql(que)
df = sdf.toPandas()

In [0]:
rows_count = sdf.count()

In [0]:
df = df.fillna(np.nan)

In [0]:
def replace_only_special_chars(text):
    if all(not i.isalnum() for i in text):
        return np.nan
    else:
        return text

In [0]:
def zeroshot_initial_analysis(df):

# '''This function calculates statistics such as the count of blank, NaN, and null values, identifies the data type and inferred data type, counts distinct and duplicate values, finds the most frequent value, and calculates length statistics. The function collects these results into a list of dictionaries, where each dictionary represents the analysis results for a specific column.'''

    # Initialize the list that will hold our analysis results
    list_of_analysis = []

    # Get the list of column names from the DataFrame
    columnNames = df.columns

    # Iterate over each column in the DataFrame
    for columnName in columnNames:
        # Initialize an empty dictionary to hold the analysis results for this column
        att_dict = {}

        # Set the attribute name in the dictionary
        att_dict["attributeName"] = columnName

        # Count the number of blank strings in the column and store it in the dictionary
        att_dict["blankCount"] = df.filter((F.col(columnName) == ' ') | (F.col(columnName) == '')).count()

        # If the column is of DoubleType, count the number of NaN values, otherwise count the number of null values
        if df.schema[columnName].dataType == DoubleType():
            att_dict["nanCount"] = df.filter(F.isnan(columnName)).count()
        else:
            att_dict["nanCount"] = df.filter(F.col(columnName).isNull()).count()

        # Count the number of null values in the column and store it in the dictionary
        att_dict["nullCount"] = df.filter(F.col(columnName).isNull() | F.col(columnName).isin(["Null", "null", "NULL", "NaT"])).count()

        # Store the data type of the column
        att_dict["attributeDataType"] = df.schema[columnName].dataType.simpleString().capitalize()

        # Store the inferred data type of the column
        att_dict["inferredDataType"] = infer_datatypes(df,columnName)['Majority Type']

        # Count the number of distinct values in the column and store it in the dictionary
        att_dict["distinctCount"] = df.select(columnName).distinct().count()

        # Create a DataFrame that contains each value in the column and the count of that value
        duplicate_df = df.groupBy(columnName).count()

        # Count the number of duplicates in the column and store it in the dictionary
        att_dict["duplicateCount"] = duplicate_df.filter(F.col('count') > 1).count()

        # Count the number of unique values in the column and store it in the dictionary
        att_dict["uniqueCount"] = duplicate_df.filter(F.col('count') == 1).count()

        # Find the most frequent value in the column and its count and store them in the dictionary
        most_freq_value = df.groupBy(columnName).count().orderBy(F.desc('count')).first()
        att_dict["mostFrequentValue"] = [str(most_freq_value[0])]
        att_dict["mostFrequentCount"] = [most_freq_value[1]]

        # Calculate the minimum, maximum, mean, and standard deviation of the length of the values in the column
        # Exclude null values from this calculation
        # Cache the result to improve performance
        lengthAnalysisDF = df.filter(F.col(columnName).isNotNull()).select(
            F.min(F.length(F.col(columnName))).alias('minLength'),
            F.max(F.length(F.col(columnName))).alias('maxLength'),
            F.avg(F.length(F.col(columnName))).alias('meanLength'),
            F.stddev(F.length(F.col(columnName))).alias('stddevLength')
        ).cache()

        firstRow = lengthAnalysisDF.first()
        att_dict["minLength"] = firstRow['minLength']
        att_dict["maxLength"] = firstRow['maxLength']
        att_dict["meanLength"] = firstRow['meanLength']
        att_dict["standaradDeviationLength"] = firstRow['stddevLength']

        lengthAnalysisDF.unpersist()

        list_of_analysis.append(att_dict)
    return list_of_analysis


In [0]:
list_of_analysis = zeroshot_initial_analysis(sdf)

In [0]:

json_filez1 = {"attributeAnalysis": list_of_analysis,
                "entityCount": len(df)} #"outputLocation": filename_zeroshot1

In [0]:
# Store outputs in differnet storage based on destination_storage_type

if destination_storage_type == 'ABS':

    blob_clientz1 = BLOB_SERVICE_CLIENT.get_blob_client(
    container=CONTAINER_NAME, blob=filename_zeroshot1)

    blob_clientz1.upload_blob(json.dumps(json_filez1, indent=2, default=myconverter).encode('utf-8'), overwrite=True)
elif destination_storage_type == 'S3':
    upload_dict_to_s3(json.dumps(json_filez1, indent=2, default=myconverter), bucket_name, filename_zeroshot1, aws_access_key_id=storage_access_key, aws_secret_access_key=storage_secret_key)
else:
    print("Invalid destination_storage_type: {destination_storage_type}")

In [0]:
# Added date info code  in Pyspark 
def replace_only_special_chars(text):
    if all(not i.isalnum() for i in text):
        return np.nan
    else:
        return text

from pyspark.sql.functions import col, udf
from pyspark.sql.types import StringType
import hidateinfer

def get_date_info(df, date_col):
    date_info_dict = {}
    
    # Clean and filter date values
    clean_date_udf = udf(lambda x: replace_only_special_chars(str(x)), StringType())
    dates = df.select(date_col).dropna().withColumn(date_col, clean_date_udf(col(date_col)))
    
    if dates.count() > 0:
        # Infer date format
        sample_frac = min(1000, rows_count)/rows_count
        sampled_dates = dates.sample(False, sample_frac, seed=42)
        date_format = hidateinfer.infer(sampled_dates.select(date_col).rdd.flatMap(lambda x: x).collect())
        date_info_dict["date_format"] = date_format
        
        # Get most frequent date
        mode = dates.groupBy(date_col).count().orderBy(col("count").desc()).first()
        most_freq_date = mode[date_col]
        cardinality = mode["count"]
        date_info_dict["most_freq_date"] = f"{most_freq_date} with cardinality = {cardinality}"
    
    return date_info_dict


In [0]:
try:
    date_cols = [column for column in sdf.columns if infer_datatypes(sdf, column)['Majority Type'] == 'Date']
    date_columns_info = {}
    for i in date_cols:
        date_info_dict = get_date_info(sdf, i)
        date_columns_info[i] = date_info_dict
except Exception as e:
    print(e)
    date_columns_info = {"date_column" : {"date_format" : "No Date column found", "most_freq_date": "No Date column found"}}


In [0]:
def character_analysis_samples(df):
    new = df.copy().fillna("").astype(str)
    col_char_samples = defaultdict(dict)
    col_char_counts = {}
    for column in df.columns:
        
        new[column + "_sp_chars"] = new[column].apply(lambda x : check_special_char(x))
        new[column + "_whitespaces"] = new[column].apply(lambda x : check_whitespaces(x))
        new[column + "_non_ascii_chars"] = new[column].apply(lambda x : check_non_ascii(x))
        new[column + "_alphanums"] = new[column].apply(lambda x : check_alnum(x))

        new[column + "_sp_chars_count"] = new[column + "_sp_chars"].map(len)
        new[column + "_whitespaces_count"] = new[column + "_whitespaces"].map(len)
        new[column + "_non_ascii_chars_count"] = new[column + "_non_ascii_chars"].map(len)
        new[column + "_alphanums_count"] = new[column + "_alphanums"].map(len)
        
        add_cols = [column + x for x in ["_sp_chars_count", "_whitespaces_count", "_non_ascii_chars_count", "_alphanums_count"]]
        new.loc[:, column+ "_all_chars_count"] = new[add_cols].sum(axis=1)
            
        for a,b in zip(["special_chars", "whitespaces", "non_ascii", "alphanum"], ["_sp_chars_count", "_whitespaces_count", "_non_ascii_chars_count", "_alphanums_count"]):
            try:
                col_char_samples[column][a] = df[new[column + b] !=0].sample(20).fillna("").to_dict("records")
            except:
                col_char_samples[column][a] = df[new[column + b] !=0].fillna("").to_dict("records")

        
        dicty = {}
        total = len(new)
        
        #sp_chars_perc = float("{:.2f}".format((sum(new[column + "_sp_chars_count"] !=0)/total)*100)) #+ "%" #np.nan_to_num(sp_chars_perc)
        #whitespaces_perc = float("{:.2f}".format((sum(new[column + "_whitespaces_count"] !=0)/total)*100))
        #non_ascii_perc = float("{:.2f}".format((sum(new[column + "_non_ascii_chars_count"] !=0)/total)*100))
        #alnum_perc = float("{:.2f}".format((sum(new[column + "_alphanums_count"] !=0)/total)*100))

        dicty["special_chars"] = sum(new[column + "_sp_chars_count"] !=0)
        dicty["whitespaces"] = sum(new[column + "_whitespaces_count"] !=0)
        dicty["non_ascii"] = sum(new[column + "_non_ascii_chars_count"] !=0)
        dicty["alphanum"] = sum(new[column + "_alphanums_count"] !=0)
        col_char_counts[column] = dicty
    
    return new, col_char_counts, col_char_samples


cols1 = list(df.columns[:int(len(df.columns)/2)])
cols2 = list(df.columns[int(len(df.columns)/2):])

char_df1, col_char_counts1, col_char_samples1 = character_analysis_samples(df[cols1])
char_df2, col_char_counts2, col_char_samples2 = character_analysis_samples(df[cols2])

col_char_counts = {**col_char_counts1, **col_char_counts2}
col_char_samples = {**col_char_samples1, **col_char_samples2}
char_df = pd.concat([char_df1, char_df2], axis=1)

In [0]:
def get_col_char_dist_samples(ddf, char_df):
    df = ddf.fillna("").astype(str)
    col_char_dists_samples = {}
    col_char_dists = defaultdict(dict)
    for column in df.columns:
 
        char_group_samples = defaultdict(dict)
        for a,b in zip(["special_chars", "whitespaces", "non_ascii", "alphanum"], ["_sp_chars", "_whitespaces", "_non_ascii_chars", "_alphanums"]):
            chars = [x for xs in char_df[column + b].to_list() for x in xs]
            non_others = [k for k, v in Counter(chars).most_common()[:11]]
            others = [k for k, v in Counter(chars).most_common()[11:]]
            
            chars_list = ["Others" if ele in others else ele for ele in chars]
            col_char_dists[column][a] = dict(Counter(chars_list))
            
            
            for char in non_others:
                try:
                    char_group_samples[a][char] = ddf[df[column].str.contains(re.escape(char))].sample(20).fillna("").to_dict("records")
                except:
                    char_group_samples[a][char] = ddf[df[column].str.contains(re.escape(char))].fillna("").to_dict("records")
                    
            try:
                char_group_samples[a]["Others"] = ddf[df[column].apply(lambda x: True if any(i in x for i in others) else False)].sample(20).fillna("").to_dict("records")
            except:  
                char_group_samples[a]["Others"] = ddf[df[column].apply(lambda x: True if any(i in x for i in others) else False)].fillna("").to_dict("records")
                    
        col_char_dists_samples[column] = dict(char_group_samples)
            
    return dict(col_char_dists), col_char_dists_samples


cols1 = list(df.columns[:int(len(df.columns)/2)])
cols2 = list(df.columns[int(len(df.columns)/2):])

col_char_dists1, col_char_dists_samples1 = get_col_char_dist_samples(df[cols1], char_df)
col_char_dists2, col_char_dists_samples2 = get_col_char_dist_samples(df[cols2], char_df)

col_char_dists= {**col_char_dists1, **col_char_dists2}
col_char_dists_samples = {**col_char_dists_samples1, **col_char_dists_samples2}

In [0]:
def getExpressionSymbolsFromCharacter(character):

    # Constants for SmartDqPattern
    smartDqDigit = 'd'
    smartDqUpper = 'A'
    smartDqLower = 'a'
    smartDqWhiteSpace = ' '
    
    
    regExDigit = "\\d+"
    regExUpper = "[A-Z]+"
    regExLower = "[a-z]+"
    regExWhiteSpace = "[\\n\\t\\r]+"
    regExSpecialChar = ".+?"

    if (character.isdigit()):
        return smartDqDigit
    elif (character.isupper()):
        return smartDqUpper
    elif (character.islower()):
        return smartDqLower
    elif (character.isspace()):
        return smartDqWhiteSpace
    else:
        return character
      
        
def PatternString(string):
    
    if string in ["nan", "NAN", "NaN", "Nan", np.nan]:
        return "NULL"
    else:
        smartDQPatternBuilder = ""
        
    for character in string:
        smartDQPatternBuilder = smartDQPatternBuilder + getExpressionSymbolsFromCharacter(character)

    return smartDQPatternBuilder

In [0]:
import plotly.express as px
import plotly.io as pio


def get_pattern_histograms_samples(ddf):
    
    df = ddf.copy().astype(str)
    pattern_jsons= {}
    pattern_plot_jsons = {}
    pattern_samples_jsons = defaultdict(dict)
    
    for column in df.columns:
        df[column] = df[column].apply(lambda x : PatternString(x))
        pattern_jsons[column] = df[column].value_counts(dropna=False).to_dict()
        
        others = list(df[~df[column].isin(df[column].value_counts().reset_index(name="count").head(10)["index"].values)][column].unique())
        df.loc[df[column].isin(others), column] = "Others"
        
        dfg = df.groupby([column]).size().to_frame().sort_values([0], ascending = False).reset_index()
        dfg.columns = [column, "count"]
        dfg["percentage"] = dfg["count"].div(dfg["count"].sum(), axis=0).multiply(100).apply(lambda x : float("{:.2f}".format(x)))
        pattern_plot_jsons[column] = dfg.to_dict("records")

        for val in dfg[column].values:
            try:
                pattern_samples_jsons[column][val] = ddf[df[column]==val].sample(20).fillna("").to_dict("records")
            except:
                pattern_samples_jsons[column][val] = ddf[df[column]==val].fillna("").to_dict("records")

    return pattern_jsons, pattern_plot_jsons, pattern_samples_jsons
    
pattern_jsons, pattern_plot_jsons, pattern_samples_jsons = get_pattern_histograms_samples(df)

In [0]:
def standardize(word):
    if "ad" in word:
        return "ad"
    elif "a" in word:
        return "a"
    elif "d" in word:
        return "d"
    else:
        return word

def get_garbage_values(ddf):
    df = ddf.copy().astype(str)
    for column in df.columns:
        df[column] = df[column].apply(lambda x : PatternString(x))
        df[column] = df[column].apply(lambda x : "".join(sorted( set( x.lower() ) ) ).replace(" ", "") )
        df[column] = df[column].apply(lambda x : standardize(x))

    df = df.replace("lnu", "NULL")
    
    pat_dict = {}
    for col in df.columns:
        pat_dict[col] = df[col].value_counts().to_dict()

     
    garbage_dict1 ={}
    for k, v in pat_dict.items():
        garbage_list1 = []
        for i in v.copy().keys():
            if i not in ["a", "ad", "d"]:
                garbage_list1.append(i)
                del v[i]
        garbage_dict1[k] = garbage_list1


    for k, v in pat_dict.items():
        s = sum(v.values())
        for x,y in v.items():
            v[x] = y * 100.0 / s


    garbage_dict2 = {}
    for k, v in pat_dict.items():
        garbage_list2 = []
        for x,y in v.items():
            if (list(v.keys())[0] =="d") and (y < 2):
                garbage_list2.append(x)
        garbage_dict2[k] = garbage_list2

    for k in garbage_dict2.copy().keys():
        garbage_dict2[k] = garbage_dict2[k] + garbage_dict1[k]

    
    garbage_vals_dict ={}
    for col in ddf.columns:
        garbage_vals_dict[col] = set(ddf[df[col].isin(garbage_dict2[col])][col].unique())
    
    return garbage_vals_dict

garbage_vals_dict = get_garbage_values(df)

In [0]:
def garb_values_dist_samples(ddf, garbage_vals_dict):
    #df = ddf.copy()
    garbage_vals_dists = defaultdict(dict)
    garbage_vals_counts = defaultdict(dict)
    garbage_vals_samples = defaultdict(dict)

    for column in ddf.columns:
        
        garb_vals = ddf[ddf[column].isin(garbage_vals_dict[column])][column].to_list()
        sorted_counts = pd.Series(garb_vals).value_counts(dropna=False).index.to_list()
        non_others = sorted_counts[:11]
        others = sorted_counts[11:]
        garbage_vals_counts[column]["garbage_values"] = len(garb_vals)
        
        garb_vals_list =["Others" if ele in others else ele for ele in garb_vals]
        garbage_vals_dists[column]["garbage_values"] = pd.Series(garb_vals_list).value_counts(dropna=False).to_dict()

        for val in non_others:
            try:
                garbage_vals_samples[column][val] = ddf[ddf[column].isin([val])].sample(20).astype(str).to_dict("records")
            except:
                garbage_vals_samples[column][val] = ddf[ddf[column].isin([val])].astype(str).to_dict("records")
        try:
            garbage_vals_samples[column]["Others"] = ddf[ddf[column].isin(others)].sample(20).astype(str).to_dict("records")
        except:
            garbage_vals_samples[column]["Others"] = ddf[ddf[column].isin(others)].astype(str).to_dict("records")
        
    return garbage_vals_counts, garbage_vals_dists, garbage_vals_samples


global splitl
def splitl(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

col_list = list(splitl(list(df.columns), 3))

garbage_vals_counts1, garbage_vals_dists1, garbage_vals_samples1 = garb_values_dist_samples(df[col_list[0]], garbage_vals_dict)
garbage_vals_counts2, garbage_vals_dists2, garbage_vals_samples2 = garb_values_dist_samples(df[col_list[1]], garbage_vals_dict)
garbage_vals_counts3, garbage_vals_dists3, garbage_vals_samples3 = garb_values_dist_samples(df[col_list[2]], garbage_vals_dict)

garbage_vals_counts = {**garbage_vals_counts1, **garbage_vals_counts2, **garbage_vals_counts3}
garbage_vals_dists = {**garbage_vals_dists1, **garbage_vals_dists2, **garbage_vals_dists3}
garbage_vals_samples = {**garbage_vals_samples1, **garbage_vals_samples2, **garbage_vals_samples3}

In [0]:
import json
key = random.sample(list(pattern_plot_jsons.keys()),1)[0]
preview_outputz2 = {"Pattern_Analysis_Plots" : pattern_plot_jsons[key]}

json_filez2 = {"Date_Columns_Info": json.dumps(date_columns_info),
             "Char_Counts": col_char_counts,
             "Gargbage_Val_Counts": dict(garbage_vals_counts), 
             "Pattern_Analysis" : [pattern_jsons], #json.dumps(pattern_dict)
             "Pattern_Analysis_Plots" : [pattern_plot_jsons]}

In [0]:
# Commented Python code for dtypes
# dtypes_samples ={}
# for i in inferred_dtypes_.keys():
#     try:
#         dtypes_samples[i] = df[inferred_dtypes_[i]].sample(20).fillna("").to_dict("records")
#     except:
#         dtypes_samples[i] = df[inferred_dtypes_[i]].fillna("").to_dict("records")

# Added Pyspark code for dtypes
all_inferred_cols = {column:infer_datatypes(sdf, column)['Majority Type'] for column in sdf.columns }

grouped_dtypes = {}
for k, v in all_inferred_cols.items():
    if v not in grouped_dtypes:
        grouped_dtypes[v] = [k]
    else:
        grouped_dtypes[v].append(k)

dtypes_samples = {}
for i in grouped_dtypes.keys():
    dtypes_samples[i] = sdf[grouped_dtypes[i]].limit(min(20, rows_count)).na.fill("").toPandas().to_dict("records")

def null_unique_distinct_analysis_samples(ddf):
    df = ddf.copy()
    null_notnull_samples = defaultdict(dict)
    unique_duplicate_samples = defaultdict(dict)
    unique_distinct_samples = defaultdict(dict)
    for column in df.columns:
        try:
            null_notnull_samples[column]["Null"] = df[df[column].isnull()].sample(20).fillna("").to_dict("records")
        except:
            null_notnull_samples[column]["Null"] = df[df[column].isnull()].fillna("").to_dict("records")

        try:
            null_notnull_samples[column]["Not_Null"] = df[df[column].notnull()].sample(20).fillna("").to_dict("records")
        except:
            null_notnull_samples[column]["Not_Null"] = df[df[column].notnull()].fillna("").to_dict("records")


        try:
            unique_duplicate_samples[column]["Unique"] = df[~df.duplicated(subset=column, keep=False)].sample(20).fillna("").to_dict("records")
            unique_distinct_samples[column]["Unique"] = df[~df.duplicated(subset=column, keep=False)].sample(20).fillna("").to_dict("records")
        except:
            unique_duplicate_samples[column]["Unique"] = df[~df.duplicated(subset=column, keep=False)].fillna("").to_dict("records")
            unique_distinct_samples[column]["Unique"] = df[~df.duplicated(subset=column, keep=False)].fillna("").to_dict("records")

        try:
            unique_duplicate_samples[column]["Duplicate"] = df[df.duplicated(subset=column, keep=False)].sample(20).fillna("").to_dict("records")
        except:
            unique_duplicate_samples[column]["Duplicate"] = df[df.duplicated(subset=column, keep=False)].fillna("").to_dict("records")
            
        try:
            unique_distinct_samples[column]["Distinct"] = df[df[column].isin(set(df[column].unique()))].sample(20).fillna("").to_dict("records")
        except:
            unique_distinct_samples[column]["Distinct"] = df[df[column].isin(set(df[column].unique()))].fillna("").to_dict("records")
            
    return null_notnull_samples, unique_duplicate_samples, unique_distinct_samples

def null_unique_distinct_analysis_samples_v2(ddf):
    df = ddf.copy()
    null_notnull_samples = defaultdict(dict)
    unique_duplicate_samples = defaultdict(dict)
    unique_distinct_samples = defaultdict(dict)
    for column in df.columns:
        try:
            null_notnull_samples[column]["Null"] = df[df[column].isnull()].sample(20).fillna("").to_dict("records")
        except:
            null_notnull_samples[column]["Null"] = df[df[column].isnull()].fillna("").to_dict("records")

        df_not_null = df[df[column].notnull()].fillna("")
        d1 = df_not_null[column].value_counts().to_dict()            
        d1 = {k:f"{k} ({v})" for k,v in d1.items()}
        df_not_null[column] = df_not_null[column].map(d1)
        null_notnull_samples[column]["Not_Null"] = df_not_null.sample(min(20, len(df_not_null))).to_dict("records")

        try:
            unique_duplicate_samples[column]["Unique"] = df[~df.duplicated(subset=column, keep=False)].sample(20).fillna("").to_dict("records")
            unique_distinct_samples[column]["Unique"] = df[~df.duplicated(subset=column, keep=False)].sample(20).fillna("").to_dict("records")
        except:
            unique_duplicate_samples[column]["Unique"] = df[~df.duplicated(subset=column, keep=False)].fillna("").to_dict("records")
            unique_distinct_samples[column]["Unique"] = df[~df.duplicated(subset=column, keep=False)].fillna("").to_dict("records")

        df_dups = df[df.duplicated(subset=column, keep=False)].fillna("")
        d1 = df_dups[column].value_counts().to_dict()            
        d1 = {k:f"{k} ({v})" for k,v in d1.items()}
        df_dups[column] = df_dups[column].map(d1)
        df_dups = df_dups.groupby(column, group_keys=True).apply(lambda x:x.head(2)).reset_index(drop=True).head(20)         
        unique_duplicate_samples[column]["Duplicate"] = df_dups.to_dict("records")
            
        try:
            unique_distinct_samples[column]["Distinct"] = df[df[column].isin(set(df[column].unique()))].sample(20).fillna("").to_dict("records")
        except:
            unique_distinct_samples[column]["Distinct"] = df[df[column].isin(set(df[column].unique()))].fillna("").to_dict("records")
            
    return null_notnull_samples, unique_duplicate_samples, unique_distinct_samples

def value_counts_dist_samples(ddf):
    df = ddf.copy()
    value_count_dist_jsons = {}
    value_count_sample_jsons = defaultdict(dict)

    for column in df.columns:
        others = list(df[~df[column].isin(df[column].value_counts().reset_index(name="count").head(10)["index"].values)][column].unique())
        df.loc[df[column].isin(others), column] = "Others"

        dfg = df.groupby([column]).size().to_frame().sort_values([0], ascending = False).reset_index()
        dfg.columns = [column, "count"]
        dfg["percentage"] = dfg["count"].div(dfg["count"].sum(), axis=0).multiply(100).apply(lambda x : float("{:.2f}".format(x)))
        value_count_dist_jsons[column] = dfg.to_dict("records")
        
        for val in dfg[column].values:
            try:
                value_count_sample_jsons[column][val] = ddf[df[column]==val].sample(20).fillna("").to_dict("records")
            except:
                value_count_sample_jsons[column][val] = ddf[df[column]==val].fillna("").to_dict("records")
    return value_count_dist_jsons, value_count_sample_jsons


def length_dist_samples(ddf):
    df = ddf.copy()
    #ddf = df.copy()
    length_dist_jsons = {}
    length_samples_jsons = defaultdict(dict)
    for column in df.columns:
        df[column + "_len_"] = df[column].fillna("").astype(str).str.len().astype(str)
        others = list(df[~df[column + "_len_"].isin(df[column + "_len_"].value_counts().reset_index(name="count").head(10)["index"].values)][column + "_len_"].unique())
        df.loc[df[column + "_len_"].isin(others), column + "_len_"] = "Others"

        dfg = df.groupby([column + "_len_"]).size().to_frame().sort_values([0], ascending = False).reset_index()
        dfg.columns = [column + "_len_", "count"]
        dfg["percentage"] = dfg["count"].div(dfg["count"].sum(), axis=0).multiply(100).apply(lambda x : float("{:.2f}".format(x)))
        length_dist_jsons[column] = dfg.to_dict("records")

        for length in dfg[column + "_len_"].values:
            #print(column, others)
            try:
                length_samples_jsons[column][length] = ddf[df[column + "_len_"]==length].sample(20).fillna("").to_dict("records")
            except:
                length_samples_jsons[column][length] = ddf[df[column + "_len_"]==length].fillna("").to_dict("records")

    return length_dist_jsons, length_samples_jsons


# d = json.loads(length_samples_jsons['first_name']['Others'])
# pd.DataFrame.from_dict(d)

null_notnull_samples_jsons, unique_duplicate_sample_jsons, unique_distinct_sample_jsons = null_unique_distinct_analysis_samples_v2(df)
value_count_dist_jsons, value_count_sample_jsons = value_counts_dist_samples(df)
length_dist_jsons, length_samples_jsons = length_dist_samples(df)

In [0]:
try:
    rec_sample = df.sample(50).fillna("").to_dict("records")
except:
    rec_sample = df.fillna("").to_dict("records")
    
zs_drill_down_outputs = {"Record_count": [rec_sample],
                     "Data_types": [dtypes_samples],
                     "Null_NotNull": [dict(null_notnull_samples_jsons)],
                     "Unique_Duplicate": [dict(unique_duplicate_sample_jsons)],
                     "Unique_Distinct" : [dict(unique_distinct_sample_jsons)],
                     "Most_Frequent": [value_count_dist_jsons, dict(value_count_sample_jsons)],
                     "Character_Analysis": [dict(col_char_dists), dict(col_char_samples), col_char_dists_samples],
                     "Garbage_Value_Analysis": [dict(garbage_vals_dists), dict(garbage_vals_samples)],
                     "Length_Analysis" : [length_dist_jsons, dict(length_samples_jsons)],
                     "Pattern_Analysis": [pattern_plot_jsons, dict(pattern_samples_jsons)] }

In [0]:
#from numpyencoder import NumpyEncoder


    
    
class NpEncoder(json.JSONEncoder):
    """ Custom encoder for numpy data types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64, np.integer)):

            return int(obj)

        elif isinstance(obj, (np.float_, np.float16, np.float32, np.float64, np.floating)):
            return float(obj)

        elif isinstance(obj, (np.complex_, np.complex64, np.complex128)):
            return {'real': obj.real, 'imag': obj.imag}

        elif isinstance(obj, (np.ndarray)):
            return obj.tolist()

        elif isinstance(obj, (np.bool_)):
            return bool(obj)

        elif isinstance(obj, (np.void)): 
            return None
        
        elif isinstance(obj, datetime.datetime):
            return obj.__str__()
        
        elif isinstance(obj, (np.generic, np.ndarray)):
            return obj.item()

        return json.JSONEncoder.default(self, obj)
    
    
class CustomJSONizer(json.JSONEncoder):
    def default(self, obj):
        return super().encode(int(obj)) \
            if isinstance(obj, np.int64) \
            else super().default(obj)

In [0]:
# Store outputs in differnet storage based on destination_storage_type

if destination_storage_type == 'ABS':

    blob_clientz2 = BLOB_SERVICE_CLIENT.get_blob_client(
        container=CONTAINER_NAME, blob=filename_zeroshot2)
    blob_clientz2.upload_blob(json.dumps(json_filez2, indent=2).encode('utf-8'), overwrite=True)

    blob_clientzp2 = BLOB_SERVICE_CLIENT.get_blob_client(
        container=CONTAINER_NAME, blob=f'{filename_zeroshot2}.preview')
    blob_clientzp2.upload_blob(json.dumps(preview_outputz2, indent=2).encode('utf-8'), overwrite=True)

    blob_clientzdd = BLOB_SERVICE_CLIENT.get_blob_client(
        container=CONTAINER_NAME, blob=f'{filename_zeroshot1}.zdd')
    blob_clientzdd.upload_blob(json.dumps(zs_drill_down_outputs, indent=2, default=myconverter, skipkeys=True).encode('utf-8'), overwrite=True) #encoding='UTF-8' inside dumps

elif destination_storage_type == 'S3':

    upload_dict_to_s3(json.dumps(json_filez2, indent=2, default=myconverter), bucket_name, filename_zeroshot2, aws_access_key_id=storage_access_key, aws_secret_access_key=storage_secret_key)

    upload_dict_to_s3(json.dumps(preview_outputz2, indent=2, default=myconverter), bucket_name, f'{filename_zeroshot2}.preview', aws_access_key_id=storage_access_key, aws_secret_access_key=storage_secret_key)

    upload_dict_to_s3(json.dumps(zs_drill_down_outputs, indent=2, default=myconverter), bucket_name, f'{filename_zeroshot1}.zdd', aws_access_key_id=storage_access_key, aws_secret_access_key=storage_secret_key)
else:
    print("Invalid destination_storage_type: {destination_storage_type}")
